# Embedding the `q` array programming language

## Syntax

In [ ]:
trait Q[R]:
    extension (i: Int)
        def int: R

    extension (f: R => R)
        def lam: R

    extension (f: (R, R) => R)
        def lam: R

    extension (q1: R)
        def til: R
        def +(q2: R): R
        def *(q2: R): R
        def apply(q2: R): R 
        def apply(q21: R, q22: R): R 
        def ++(q2: R): R
        def cross(q2: R): R
        def `#`(l: R): R
        def count: R
        def reverse: R
        def |>(f: R => R): R
        def drop(i: R): R

    def min: R 
    def `*`: R

    extension (f: (R, R) => R)
        def each: R = 
            ((p: R) => f(p(0.int), p(1.int))).lam.each

    extension (f: R)
        def each: R
        def over: R

object Q: 
    def apply[R](using Q: Q[R]) = Q

    object syntax: 
        def `*`[R](using Q: Q[R]): R = Q.`*`

## Programs

In [ ]:
// {(*/)1+til x}
import Q.syntax._

def factorial[R: Q] = 
    {(x: R) => `*`.over(1.int + x.til)}.lam

In [ ]:
// {(min'')(2#count w)#w cross w:v,1_reverse v:1+til x}

def pyramid[R: Q] = 
    {(x: R) =>
        (1.int + x.til) |> { v => 
            (v ++ v.reverse.drop(1.int)) |> { w => 
                 Q[R].min.each.each(2.int `#` w.count `#` (w cross w))
            }
        }
    }.lam

## Q serialization

In [ ]:
import $ivy.`org.typelevel::cats-core:2.10.0`

In [ ]:
import cats._, cats.data._, cats.syntax.all._, cats.implicits._

In [ ]:
case class S(i: Int, b: Boolean, m: Map[Int, String])

object S: 
    type QSerA[A] = State[S, A]
    type QSer = State[S, String]

    def paren(s: String)(b: Boolean): String = 
        if (b) s"($s)"
        else s

    def getParen: QSerA[Boolean] = 
        State.inspect:
            case S(i, b, s) => b

    def getVar: QSerA[Int] = 
        State.inspect:
            case S(i, b, s) => i

    def getVarValue(i: Int): QSer = 
        State.inspect:
            case S(_, _, s) => s(i)

    def incVar: QSerA[Unit] = 
        State.modify:
            case S(i, b, m) => S(i+1, b, m)

    def modifyMap(f: Map[Int, String] => Map[Int, String]): QSerA[Unit] = 
        State.modify:
            case S(i, b, m) => S(i, b, f(m))

    def setValue(i: Int, s: String): QSerA[Unit] = 
        modifyMap(_ + (i -> s))

    def getValue(i: Int): QSer = 
        for
            v <- getVarValue(i)
            _ <- setValue(i, s"x$i")
        yield v

    extension (q: QSer)
        def withParen(b: Boolean): QSer =
            State.modify[S]{ case S(i, _, s) => S(i, b, s) } *> q 

        def serialize: String = 
            q.runA(S(0, false, Map())).value



In [ ]:
import S._

object QSer extends Q[QSer]:

    extension (i: Int)
        def int: QSer = State.pure(i.toString)

    extension (f: QSer => QSer)
        def lam: QSer = for
            i1 <- f(State.pure(s"x")).withParen(false)
        yield s"{$i1}"

    extension (f: (QSer, QSer) => QSer)
        def lam: QSer = for
            i1 <- f(State.pure("x"), State.pure("y")).withParen(false)
        yield s"{$i1}"

    extension (q1: QSer)

        def op2(op: String, sp: String)(q2: QSer): QSer = for
            b <- getParen
            i2 <- q2.withParen(false)
            i1 <- q1.withParen(true)
        yield paren(s"$i1$sp$op$sp$i2")(b)

        def op1(op: String): QSer = for
            b <- getParen
            i1 <- q1.withParen(false)
        yield paren(s"$op $i1")(b)
                 
        def op1post(op: String): QSer = for
            b <- getParen
            i1 <- q1.withParen(false)
        yield paren(s"$i1$op")(b)
                 
        def til: QSer = q1.op1("til")

        def +(q2: QSer): QSer = q1.op2("+", "")(q2)

        def *(q2: QSer): QSer = q1.op2("*", "")(q2)

        def apply(q2: QSer): QSer  = for
            b <- getParen
            i1 <- q1.withParen(true)
            i2 <- q2.withParen(false)
        yield paren(s"$i1$i2")(b)

        def apply(q21: QSer, q22: QSer): QSer = for
            b <- getParen
            i1 <- q1.withParen(true)
            i21 <- q21.withParen(true)
            i22 <- q22.withParen(false)
        yield paren(s"$i1[$i21,$i22]")(b)

        def ++(q2: QSer): QSer = q1.op2(",", "")(q2)

        def cross(q2: QSer): QSer = q1.op2("cross", " ")(q2)

        def `#`(q2: QSer): QSer = q1.op2("#", "")(q2)

        def count: QSer = q1.op1("count")

        def reverse: QSer = q1.op1("reverse")

        def |>(f: QSer => QSer): QSer = for
            b <- getParen
            i <- getVar
            i1 <- q1.withParen(false)
            _ <- setValue(i, s"x$i:$i1")
            _ <- incVar
            i2 <- f(getValue(i)).withParen(false)
        yield paren(i2)(b)

        def drop(q2: QSer): QSer = q2.op2(" _ ", "")(q1)

    def min: QSer  = State.pure("min")
    def `*`: QSer  = State.pure("*")

    extension (q1: QSer)
        def each: QSer = q1.op1post("'")
        def over: QSer = q1.op1post("/")


In [ ]:
factorial(using QSer).serialize

In [ ]:
def p1[R: Q]: R = 
    3.int.til + 1.int + 5.int

In [ ]:
p1(using QSer).serialize

In [ ]:
// {(min'')(2#count w)#w cross w:v,1_reverse v:1+til x}

pyramid(using QSer).serialize

# References

* kdb+/q: https://kx.com/developers/